In [1]:
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

C:\Users\piyus\Anaconda3\envs\tf-env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
print(os.listdir("C:/Users/piyus/Downloads/Downloads1/Dataset/validation/validation/"))
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import gc

['0001eeaf4aed83f9.jpg', '0004886b7d043cfd.jpg', '000595fe6fee6369.jpg', '00075905539074f2.jpg', '0007cebe1b2ba653.jpg', '0007d6cf88afaa4a.jpg', '0008e425fb49a2bf.jpg', '0009bad4d8539bb4.jpg', '000a045a0715d64d.jpg', '000a1249af2bc5f0.jpg', '000ada55d36b4bcb.jpg', '000c4d66ce89aa69.jpg', '00101a0160a05d31.jpg', '001083f05db4352b.jpg', '0010c714a5da358a.jpg', '0013e81caff9a7b2.jpg', '0013ea2087020901.jpg', '00141571d986d241.jpg', '001464cfae2a30b8.jpg', '00146ba1e50ed8d8.jpg', '00173afdc7581c41.jpg', '001840a807e454c7.jpg', '0019308d876736fe.jpg', '001997021f01f208.jpg', '0019e544c79847f5.jpg', '001a78754e43abc5.jpg', '001a794d1865ee47.jpg', '001a809ad40a2f84.jpg', '001a995c1e25d892.jpg', '001ffaceaff5f33f.jpg', '0022bffa9abfb554.jpg', '0022d0ab1e1347ab.jpg', '0022e32008e479cb.jpg', '0022ecd6f681bed6.jpg', '00230c216711fceb.jpg', '002aab1d644cae0e.jpg', '002b0dc60b1068a9.jpg', '002cfe2087f432e0.jpg', '002d1dd67c722d98.jpg', '002f8241bd829022.jpg', '003132be85a0beae.jpg', '0032257bf3cd56

In [3]:
df = pd.read_csv('C:/Users/piyus/Downloads/Downloads1/Dataset/validation-annotations-human-imagelabels.csv', usecols=[0,2,3])

In [4]:
df = df[df.Confidence == 1]

In [17]:
import numpy  as np

In [22]:
classes = np.array(['/m/01g317', '/m/09j2d', '/m/04yx4', '/m/0dzct', '/m/07j7r', '/m/05s2s', '/m/03bt1vf', '/m/07yv9', '/m/0cgh4', '/m/01prls', '/m/09j5n', '/m/0jbk', '/m/0k4j', '/m/05r655', '/m/02wbm', '/m/0c9ph5', '/m/083wq', '/m/0c_jw', '/m/03jm5', '/m/0d4v4'])

In [23]:
li = []
for i in classes:
    li.append(df[df.LabelName == i])

In [8]:
df = pd.concat(li).sample(frac=1).reset_index(drop=True)

In [9]:
del li
gc.collect()
df.head()

,ImageID,LabelName,Confidence
0,e9ed6fdef88b0c30,/m/09j2d,1
1,4495749523ee5325,/m/07j7r,1
2,95dcf58de4233ec8,/m/083wq,1
3,133921bd26fff729,/m/09j2d,1
4,0ada35baba28134b,/m/07yv9,1


In [10]:
labels = df.LabelName.tolist()
Imageid = df.ImageID.values
print(len(df))

68867


In [11]:
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model, Sequential
from tqdm import tqdm_notebook
from tqdm import tqdm
from keras.optimizers import Adam

In [12]:
import sys 
from PIL import Image 

In [13]:
X_train = [np.array(load_img(r'C:\Users\piyus\Downloads\Downloads1\Dataset\validation\validation\{}.jpg'.format(i),target_size=(139,139), grayscale=False))/255 for i in tqdm(Imageid[100:200])]

100%|██████████| 100/100 [00:07<00:00, 12.87it/s]


In [14]:
X_Val = [np.array(load_img(r'C:\Users\piyus\Downloads\Downloads1\Dataset\validation\validation\{}.jpg'.format(i),target_size=(139,139), grayscale=False))/255 for i in tqdm(Imageid[:20])]

100%|██████████| 20/20 [00:02<00:00,  9.30it/s]


In [24]:
classes = classes.tolist()
Y_train, Y_Val = [], []
for i in tqdm(labels[100:200]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_train.append(temp)
    del temp
for i in tqdm(labels[:20]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_Val.append(temp)
    del temp
Y_train = np.array(Y_train)
Y_Val = np.array(Y_Val)
gc.collect(), Y_train[0]

100%|██████████| 20/20 [00:00<00:00, 18716.22it/s]


(271,
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [31]:
myModel = InceptionResNetV2(weights='imagenet', include_top=False,input_shape = (139,139,3))
myModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 139, 139, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 69, 69, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 69, 69, 32)   96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 69, 69, 32)   0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

__________________________________________________________________________________________________
mixed_5b (Concatenate)          (None, 15, 15, 320)  0           activation_209[0][0]             
                                                                 activation_211[0][0]             
                                                                 activation_214[0][0]             
                                                                 activation_215[0][0]             
__________________________________________________________________________________________________
conv2d_219 (Conv2D)             (None, 15, 15, 32)   10240       mixed_5b[0][0]                   
__________________________________________________________________________________________________
batch_normalization_219 (BatchN (None, 15, 15, 32)   96          conv2d_219[0][0]                 
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
activation_222 (Activation)     (None, 15, 15, 32)   0           batch_normalization_222[0][0]    
__________________________________________________________________________________________________
activation_224 (Activation)     (None, 15, 15, 32)   0           batch_normalization_224[0][0]    
__________________________________________________________________________________________________
activation_227 (Activation)     (None, 15, 15, 64)   0           batch_normalization_227[0][0]    
__________________________________________________________________________________________________
block35_2_mixed (Concatenate)   (None, 15, 15, 128)  0           activation_222[0][0]             
                                                                 activation_224[0][0]             
                                                                 activation_227[0][0]             
__________

__________________________________________________________________________________________________
conv2d_234 (Conv2D)             (None, 15, 15, 32)   10240       block35_3_ac[0][0]               
__________________________________________________________________________________________________
conv2d_236 (Conv2D)             (None, 15, 15, 32)   9216        activation_235[0][0]             
__________________________________________________________________________________________________
conv2d_239 (Conv2D)             (None, 15, 15, 64)   27648       activation_238[0][0]             
__________________________________________________________________________________________________
batch_normalization_234 (BatchN (None, 15, 15, 32)   96          conv2d_234[0][0]                 
__________________________________________________________________________________________________
batch_normalization_236 (BatchN (None, 15, 15, 32)   96          conv2d_236[0][0]                 
__________

__________________________________________________________________________________________________
conv2d_247 (Conv2D)             (None, 15, 15, 32)   10240       block35_5_ac[0][0]               
__________________________________________________________________________________________________
conv2d_250 (Conv2D)             (None, 15, 15, 48)   13824       activation_249[0][0]             
__________________________________________________________________________________________________
batch_normalization_247 (BatchN (None, 15, 15, 32)   96          conv2d_247[0][0]                 
__________________________________________________________________________________________________
batch_normalization_250 (BatchN (None, 15, 15, 48)   144         conv2d_250[0][0]                 
__________________________________________________________________________________________________
activation_247 (Activation)     (None, 15, 15, 32)   0           batch_normalization_247[0][0]    
__________

...

mixed_7a (Concatenate)          (None, 3, 3, 2080)   0           activation_361[0][0]             
                                                                 activation_363[0][0]             
                                                                 activation_366[0][0]             
                                                                 max_pooling2d_8[0][0]            
__________________________________________________________________________________________________
conv2d_368 (Conv2D)             (None, 3, 3, 192)    399360      mixed_7a[0][0]                   
__________________________________________________________________________________________________
batch_normalization_368 (BatchN (None, 3, 3, 192)    576         conv2d_368[0][0]                 
__________________________________________________________________________________________________
activation_368 (Activation)     (None, 3, 3, 192)    0           batch_normalization_368[0][0]    
__________

__________________________________________________________________________________________________
activation_377 (Activation)     (None, 3, 3, 224)    0           batch_normalization_377[0][0]    
__________________________________________________________________________________________________
conv2d_375 (Conv2D)             (None, 3, 3, 192)    399360      block8_2_ac[0][0]                
__________________________________________________________________________________________________
conv2d_378 (Conv2D)             (None, 3, 3, 256)    172032      activation_377[0][0]             
__________________________________________________________________________________________________
batch_normalization_375 (BatchN (None, 3, 3, 192)    576         conv2d_375[0][0]                 
__________________________________________________________________________________________________
batch_normalization_378 (BatchN (None, 3, 3, 256)    768         conv2d_378[0][0]                 
__________

__________________________________________________________________________________________________
block8_5_mixed (Concatenate)    (None, 3, 3, 448)    0           activation_383[0][0]             
                                                                 activation_386[0][0]             
__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, 3, 3, 2080)   933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, 3, 3, 2080)   0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, 3, 3, 2080)   0           block8_5[0][0]                   
__________

__________________________________________________________________________________________________
activation_396 (Activation)     (None, 3, 3, 192)    0           batch_normalization_396[0][0]    
__________________________________________________________________________________________________
conv2d_397 (Conv2D)             (None, 3, 3, 224)    129024      activation_396[0][0]             
__________________________________________________________________________________________________
batch_normalization_397 (BatchN (None, 3, 3, 224)    672         conv2d_397[0][0]                 
__________________________________________________________________________________________________
activation_397 (Activation)     (None, 3, 3, 224)    0           batch_normalization_397[0][0]    
__________________________________________________________________________________________________
conv2d_395 (Conv2D)             (None, 3, 3, 192)    399360      block8_7_ac[0][0]                
__________

__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 3, 3, 256)    768         conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_403 (Activation)     (None, 3, 3, 192)    0           batch_normalization_403[0][0]    
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 3, 3, 256)    0           batch_normalization_406[0][0]    
__________________________________________________________________________________________________
block8_10_mixed (Concatenate)   (None, 3, 3, 448)    0           activation_403[0][0]             
                                                                 activation_406[0][0]             
__________________________________________________________________________________________________
block8_10_

In [32]:
x = myModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(100, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

In [33]:
model = Model(inputs=myModel.input, outputs=predictions)

In [77]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 139, 139, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 69, 69, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 69, 69, 32)   96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 69, 69, 32)   0           batch_normalization_377[0][0]    
__________________________________________________________________________________________________
conv2d_378

__________________________________________________________________________________________________
batch_normalization_396 (BatchN (None, 15, 15, 64)   192         conv2d_396[0][0]                 
__________________________________________________________________________________________________
batch_normalization_398 (BatchN (None, 15, 15, 64)   192         conv2d_398[0][0]                 
__________________________________________________________________________________________________
batch_normalization_401 (BatchN (None, 15, 15, 96)   288         conv2d_401[0][0]                 
__________________________________________________________________________________________________
batch_normalization_402 (BatchN (None, 15, 15, 64)   192         conv2d_402[0][0]                 
__________________________________________________________________________________________________
activation_396 (Activation)     (None, 15, 15, 64)   0           batch_normalization_396[0][0]    
__________

__________________________________________________________________________________________________
batch_normalization_417 (BatchN (None, 7, 7, 192)    576         conv2d_417[0][0]                 
__________________________________________________________________________________________________
batch_normalization_420 (BatchN (None, 7, 7, 192)    576         conv2d_420[0][0]                 
__________________________________________________________________________________________________
batch_normalization_425 (BatchN (None, 7, 7, 192)    576         conv2d_425[0][0]                 
__________________________________________________________________________________________________
batch_normalization_426 (BatchN (None, 7, 7, 192)    576         conv2d_426[0][0]                 
__________________________________________________________________________________________________
activation_417 (Activation)     (None, 7, 7, 192)    0           batch_normalization_417[0][0]    
__________

batch_normalization_450 (BatchN (None, 7, 7, 192)    576         conv2d_450[0][0]                 
__________________________________________________________________________________________________
activation_450 (Activation)     (None, 7, 7, 192)    0           batch_normalization_450[0][0]    
__________________________________________________________________________________________________
conv2d_447 (Conv2D)             (None, 7, 7, 192)    147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_451 (Conv2D)             (None, 7, 7, 192)    258048      activation_450[0][0]             
__________________________________________________________________________________________________
batch_normalization_447 (BatchN (None, 7, 7, 192)    576         conv2d_447[0][0]                 
__________________________________________________________________________________________________
batch_norm

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_10[0][0]             
                                                                 activation_470[0][0]             
__________________________________________________________________________________________________
global_average_pooling2d_10 (Gl (None, 2048)         0           mixed10[0][0]                    
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 100)          204900      global_average_pooling2d_10[0][0]
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 20)           2020        dense_14[0][0]                   
Total params: 22,009,704
Trainable params: 21,975,272
Non-trainable params: 34,432
__________________________

In [34]:
X_train = np.array(X_train).reshape((100,139,139,3))
X_Val = np.array(X_Val).reshape((20,139,139,3))

358943

In [35]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

In [36]:
plotinfo=model.fit(X_train, Y_train, validation_data=(X_Val,Y_Val), batch_size=50, epochs=10, verbose=1)

Train on 100 samples, validate on 20 samples
Epoch 1/10
100/100 [==============================] - ETA: 55s - loss: 3.1953 - acc: 0.0000e+ - 85s 852ms/step - loss: 3.3019 - acc: 0.0600 - val_loss: 8.3218 - val_acc: 0.0500
Epoch 2/10
100/100 [==============================] - ETA: 23s - loss: 2.8257 - acc: 0.20 - 49s 492ms/step - loss: 2.7751 - acc: 0.2100 - val_loss: 9.1467 - val_acc: 0.0500
Epoch 3/10
100/100 [==============================] - ETA: 24s - loss: 3.1988 - acc: 0.26 - 53s 530ms/step - loss: 3.3215 - acc: 0.2600 - val_loss: 3.3906 - val_acc: 0.0500
Epoch 4/10
100/100 [==============================] - ETA: 25s - loss: 2.5191 - acc: 0.44 - 52s 524ms/step - loss: 2.3226 - acc: 0.4400 - val_loss: 3.0282 - val_acc: 0.1000
Epoch 5/10
100/100 [==============================] - ETA: 24s - loss: 1.3873 - acc: 0.66 - 52s 515ms/step - loss: 1.7913 - acc: 0.6200 - val_loss: 2.9794 - val_acc: 0.1000
Epoch 6/10
100/100 [==============================] - ETA: 25s - loss: 1.3485 - acc: 0